In [ ]:
%pip install -r requirements.txt

In [ ]:
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta
from data_gathering import get_traffic, get_active_cameras_by_segment, get_all_cameras

## Parameters

Get all the segments for which the two versions operate simultaneously

In [ ]:
cameras = get_all_cameras()
cameras_per_segments = cameras.groupby('segment_id')['hardware_version'].nunique()
cameras_per_segments.sort_values(ascending=False)[0:10]

In [ ]:
burel = 9000002156
yesterday = (datetime.now() - timedelta(2))

cameras = get_active_cameras_by_segment(burel)
start_period = "2023-09-08 00:00:00Z"
end_period = yesterday.strftime('%Y-%m-%d %H:%M:%SZ')

## Get Telraam data 

In [ ]:
burel = pd.DataFrame()
for version in cameras:
    burel_tmp = get_traffic(cameras[version], start_period, end_period, level='instances')
    burel_tmp['version'] = version
    burel = pd.concat([burel, burel_tmp], ignore_index=True)

#burel.to_csv(f"data/burel_{yesterday.strftime('%Y%m%d')}.csv", index=False)

In [ ]:
burel.groupby('version')['version'].count()

In [ ]:
burel.columns

In [ ]:
burel.tail()

## Analysis

Ideas :
- daily aggregation and comparison 
    - global (or just cars + heavy)
    - by transportation mode
    - by directions
- by moment of time :
    - day of week
    - hour
- uptime differences ?
- v85 differences ?
- directions

In [ ]:
#burel = pd.read_csv('data/burel_20231018.csv')

burel['date'] = pd.to_datetime(burel['date'])
burel['day'] = burel['date'].dt.date
burel['hour'] = burel['date'].dt.hour

dow_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
burel['day_of_week'] = burel['date'].dt.strftime('%A')
burel['day_of_week'] = pd.Categorical(burel['day_of_week'], categories=dow_order)


burel = burel.query('uptime > 0.5')

### Uptime

In [ ]:
id_var = ['day','version']
median_uptime = burel.groupby(id_var)['uptime'].median().reset_index()
sns.lineplot(median_uptime, x="day", y="uptime", hue="version", errorbar=None)

In [ ]:
sns.boxplot(burel.query("version == 'v2'")['uptime']) 

### Daily - by Transportation mode

In [ ]:
id_var = ['day','version']
evolution = burel.groupby(id_var)[transportation_modes].sum().reset_index()
evolution = pd.melt(evolution, id_vars=id_var, var_name="mode")

In [ ]:
g = sns.FacetGrid(evolution, col="mode", sharey=False, col_wrap=2)
g.map_dataframe(sns.lineplot, x='day', y='value', hue='version', errorbar=None)
g.add_legend()

### Daily - by transportation mode and direction

In [ ]:
evolution_direction = burel.groupby(id_var)[direction_modes].sum().reset_index()
evolution_direction['vehicules_lft'] = evolution_direction['car_lft'] + evolution_direction['heavy_lft']
evolution_direction['vehicules_rgt'] = evolution_direction['car_rgt'] + evolution_direction['heavy_rgt']
evolution_direction = pd.melt(evolution_direction, id_vars=id_var, var_name="mode_direction")
evolution_direction[['mode', 'direction']] = evolution_direction['mode_direction'].str.split("_", expand = True)


In [ ]:
g = sns.FacetGrid(evolution_direction, col="mode", row="direction", sharey=False)
g.map_dataframe(sns.lineplot, x='day', y='value', hue='version', errorbar=None)
g.add_legend()

In [ ]:
evolution_direction

### Day of week

In [ ]:
id_var = ['day_of_week','version']
dow_direction = burel.groupby(id_var, observed=False)[direction_modes].median().reset_index()
dow_direction = pd.melt(dow_direction, id_vars=id_var, var_name="mode_direction")
dow_direction[['mode', 'direction']] = dow_direction['mode_direction'].str.split("_", expand = True)


In [ ]:
g = sns.FacetGrid(dow_direction, col="mode", row="direction", sharey=False)
g.map_dataframe(sns.lineplot, x='day_of_week', y='value', hue='version', errorbar=None, sort=False)
g.add_legend()

 ### Hourly

In [ ]:
id_var = ['hour','version']
hour_direction = burel.groupby(id_var, observed=False)[direction_modes].median().reset_index()
hour_direction = pd.melt(hour_direction, id_vars=id_var, var_name="mode_direction")
hour_direction[['mode', 'direction']] = hour_direction['mode_direction'].str.split("_", expand = True)

In [ ]:
g = sns.FacetGrid(hour_direction, col="mode", row="direction", sharey=False)
g.map_dataframe(sns.lineplot, x='hour', y='value', hue='version', errorbar=None, sort=False)
g.add_legend()